# <b> 1. LSA(Latent Semantic Analysis)</b> 

In [6]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import TreebankWordTokenizer

In [7]:
df_2000 = pd.read_csv('../유행어_전처리데이터/2000유행어_전처리.csv', encoding='utf-8')
df_2020 = pd.read_csv('../유행어_전처리데이터/20유행어_전처리.csv', encoding='utf-8')
df_2021 = pd.read_csv('../유행어_전처리데이터/21유행어_전처리.csv', encoding='utf-8')
df_2022 = pd.read_csv('../유행어_전처리데이터/22유행어_전처리.csv', encoding='utf-8')
df_2023 = pd.read_csv('../유행어_전처리데이터/23유행어_전처리.csv', encoding='utf-8')
df_2024 = pd.read_csv('../유행어_전처리데이터/24유행어_전처리.csv', encoding='utf-8')


df = pd.concat([df_2000, df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True)
df.head()

,postdate,body,description,title,preprocessed_body,preprocessed_description,preprocessed_title
0,20230310,안녕하세요 디벨롭스토리즈입니다 시대의 변화에 따라 새로운 것들을 표현하기 위해 새롭...,신조어들 2000년대에도 신조어가 존재했다는데요 Y2K 시리즈 그 시절 유행어에 대...,2000년대에도 신조어가 있었다 Y2K 유행 파헤쳐 보기 그 시절 옛날 유행어 지대...,안녕하세요 디벨 롭 스토리 기 사전 정의 데 생성 말 중 엽기 지대 은 위 뜻 지 ...,중 엽기 지대 은 위,도 보기 옛날 지대 엽기 즐 뜻
1,20190607,스마트 친구들 안녕 곧 주말이라 신나는 이 시간 지난 5월 27일부터 6월 2일 까...,지난 5월 27일부터 6월 2일 까지 진행한 b2000년대 유행어b 맞주기 이벤트 ...,당첨자 발표 2000년대 유행어 맞추기 이벤트,스마트 주말 라 지나 맞주기 이벤트 당첨자 퀴즈 이벤트 주 답 좌절 습기 고고씽 제...,지나 맞주기 이벤트 당첨자 퀴즈 이벤트 주,당첨자 이벤트
2,20231222,여러분 안녕하세요 그린랜드입니다 오늘은 2023년을 마무리하며 올한해 일본의 신조어...,그린랜드입니다 오늘은 2023년을 마무리하며 올한해 일본의 신조어b유행어b를 Y2K...,2023년 일본 신조어유행어 TOP 10 후보 리스트,그린랜드 은 마무리 해 도 수 알 아레 아레 일본어 거 뜻 요 야구 구단 한신 타이...,그린랜드 은 마무리 해 복장,후보 리스트
3,20220502,안녕하세요 시사중국어사 24기 기자단 신혜빈입니다 여러분은 평소에 줄임말 자주 쓰시...,오늘은 중국의 MZ세대가 쓰는 중국어 b유행어b에는 어떤 것이 있는지 또 어떤 줄임...,이것만 알면 나도 핵인싸 중국의 MZ세대 줄임말과 유행어,안녕하세요 시사 중 국어사 기 기자단 신혜 빈 은 평소 줄 말 유학 더치페이 말 계...,은 중국어 밀레 얼 제트 초 초,만 핵 인 말
4,20200316,본문을 찾을 수 없습니다,httpskrtheepochtimescomeb909cec9ea5eb8580c2b7e...,제목을 찾을 수 없습니다,수,NaN,수


In [10]:
# 결측치 제거
df = df.dropna(subset=['preprocessed_body'])
df.shape

(5624, 7)

### 유행어만 남김

In [11]:
# 유행어 텍스트 파일에서 유행어 읽기
def load_inwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        inwords_list = file.read().splitlines()
    return inwords_list

# 파일에서 유행어 목록 로드
inwords_file_path = '../../new.txt'
file_inwords = load_inwords(inwords_file_path)

# 유행어 리스트 중복 삭제
all_inwords = list(set(file_inwords))

In [12]:
def filter_inwords(text, inwords):
    words = text.split()
    filtered_words = [word for word in words if word in inwords]
    return ' '.join(filtered_words)

# preprocessed_body 칼럼에서 유행어만 남기기
df['preprocessed_inwords'] = df['preprocessed_body'].apply(lambda x: filter_inwords(x, all_inwords))

df.head()

,postdate,body,description,title,preprocessed_body,preprocessed_description,preprocessed_title,preprocessed_inwords
0,20230310,안녕하세요 디벨롭스토리즈입니다 시대의 변화에 따라 새로운 것들을 표현하기 위해 새롭...,신조어들 2000년대에도 신조어가 존재했다는데요 Y2K 시리즈 그 시절 유행어에 대...,2000년대에도 신조어가 있었다 Y2K 유행 파헤쳐 보기 그 시절 옛날 유행어 지대...,안녕하세요 디벨 롭 스토리 기 사전 정의 데 생성 말 중 엽기 지대 은 위 뜻 지 ...,중 엽기 지대 은 위,도 보기 옛날 지대 엽기 즐 뜻,엽기 지대 즐 즐 즐 엽기 엽기 엽기 엽기 지대 지대 지대 지대 지대 레알 엽기 지...
1,20190607,스마트 친구들 안녕 곧 주말이라 신나는 이 시간 지난 5월 27일부터 6월 2일 까...,지난 5월 27일부터 6월 2일 까지 진행한 b2000년대 유행어b 맞주기 이벤트 ...,당첨자 발표 2000년대 유행어 맞추기 이벤트,스마트 주말 라 지나 맞주기 이벤트 당첨자 퀴즈 이벤트 주 답 좌절 습기 고고씽 제...,지나 맞주기 이벤트 당첨자 퀴즈 이벤트 주,당첨자 이벤트,고고씽
2,20231222,여러분 안녕하세요 그린랜드입니다 오늘은 2023년을 마무리하며 올한해 일본의 신조어...,그린랜드입니다 오늘은 2023년을 마무리하며 올한해 일본의 신조어b유행어b를 Y2K...,2023년 일본 신조어유행어 TOP 10 후보 리스트,그린랜드 은 마무리 해 도 수 알 아레 아레 일본어 거 뜻 요 야구 구단 한신 타이...,그린랜드 은 마무리 해 복장,후보 리스트,문제 문제 문제 문제 문제 문제
3,20220502,안녕하세요 시사중국어사 24기 기자단 신혜빈입니다 여러분은 평소에 줄임말 자주 쓰시...,오늘은 중국의 MZ세대가 쓰는 중국어 b유행어b에는 어떤 것이 있는지 또 어떤 줄임...,이것만 알면 나도 핵인싸 중국의 MZ세대 줄임말과 유행어,안녕하세요 시사 중 국어사 기 기자단 신혜 빈 은 평소 줄 말 유학 더치페이 말 계...,은 중국어 밀레 얼 제트 초 초,만 핵 인 말,
4,20200316,본문을 찾을 수 없습니다,httpskrtheepochtimescomeb909cec9ea5eb8580c2b7e...,제목을 찾을 수 없습니다,수,NaN,수,


In [13]:
# 상위 (max_features)개의 단어에 대해서만 vectorize  -> 각각의 문서들을 벡터 형태로 만들기 위해서 사용
# max_df=0.5: 문서의 절반 이상에서 나타나는 단어를 무시 -> 자주 등장하지만 정보량이 적은 단어 
vectorizer = TfidfVectorizer(max_features=100, max_df=0.5, smooth_idf=True) 
X = vectorizer.fit_transform(df['preprocessed_inwords'])

In [14]:
print(X.shape)

(5624, 100)


In [15]:
# X는 sparse matrix. 
# 효율적인 저장 방식인 CSR(Compressed Sparse Row) matrix 형식으로 저장되어 있음
X

<5624x100 sparse matrix of type '<class 'numpy.float64'>'
	with 7126 stored elements in Compressed Sparse Row format>

- <span style = 'font-size:1.1em;line-height:1.5em'>TF-IDF의 각 차원의 의미</span>

In [16]:
terms = vectorizer.get_feature_names_out()
print(terms) # 벡터의 각 차원이 어떤 단어를 의미하는지 
print(len(terms)) # 총 벡터의 차원은 100차원 

['간지' '감성' '갑툭튀' '거북이' '겠냐' '고고씽' '고양이' '고진감래' '궁금' '귀여니' '기절' '꼬마' '꾸웨액'
 '꾸웨엑' '낭만' '너뭐돼' '농협은행' '뉴진스' '당모치' '대박' '댕댕이' '동료' '디토' '띵작' '레게노' '레알'
 '루돌프' '루삥뽕' '만남' '맑눈광' '맛꿀마' '머선' '먹방' '모에' '무야호' '문제' '뭥미' '부캐' '분좋카'
 '브라보' '빙고' '빵꾸똥꾸' '삼귀다' '소고기' '손민수' '쉽살재빙' '스라밸' '스불재' '신나' '신뢰' '쌉파서블'
 '아가씨' '아메리카노' '아이스' '알잘딱깔센' '어사' '어쩔티비' '억텐' '얼죽아' '얼짱' '엄마' '여신' '연진'
 '엽기' '오운완' '오이시쿠나레' '오타쿠' '우왕' '원영' '이유' '이제' '인성' '일취월장' '잡채' '재질' '잼민'
 '잼민이' '저메추' '저쩔' '점메추' '존귀' '졸귀' '중꺽마' '중꺾마' '지대' '지못미' '쫌쫌따리' '찐텐' '초딩'
 '추구미' '축복' '캘박' '커피' '킹받네' '킹받드라슈' '킹왕짱' '하이' '한강' '혜정' '홍대']
100


- <span style = 'font-size:1.1em;line-height:1.5em'>TF-IDF matrix에 Truncated SVD를 적용하여 LSA 수행</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>n_components = 토픽 수</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>일반적으로 TF-IDF 특징 수의 5%에서 20% 사이의 값을 토픽 수로 설정
    
#### **연도별 데이터 프레임을 합쳤기 때문에 각 연도별로 유의미한 차이가 있을 것이라고 생각하고 토픽 수를 6개로 진행**

In [17]:
model = TruncatedSVD(n_components=6, n_iter=100, random_state=42)  
## 연도별 데이터프레임을 합쳤기 때문에 2000년대, 20년, 21년, 22년, 23년, 24년으로 나눔
model.fit(X)

TruncatedSVD(n_components=6, n_iter=100, random_state=42)

$$X=U \Sigma V^{T} \approx U_{trunc} \Sigma_{trunc} V_{trunc}^{T}$$ 
- <span style = 'font-size:1.2em;line-height:1.5em'>여기서 <code>result=model.transform(X)</code>: $U_{trunc} \Sigma_{trunc}$</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>여러 문서를 특정 차원의 벡터로 (즉, n개 단어의 조합으로) 표현</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>그러나, SVD를 활용하여 여러 문서를 특정 차원의 벡터로 (즉, n개 주제의 조합으로) 표현 </span>

In [18]:
result = model.transform(X)
print(result.shape)

(5624, 6)


In [19]:
result.shape

(5624, 6)

In [20]:
# 각 topic이 몇 개의 단어 조합으로 되어있는지
print(model.components_.shape)
print(model.components_)

(6, 100)
[[ 9.79834459e-04  3.87823872e-02  9.12610669e-04  3.30312785e-03
   4.66242113e-05  9.87947617e-04  2.19301706e-02  4.13873434e-04
   4.13036802e-02  1.11326691e-04  4.78443811e-03  3.75903669e-03
   3.95071184e-04  4.25725178e-05  8.23022754e-03  4.43684784e-05
   1.53935997e-03  3.29012875e-03  1.04660682e-03  5.27781638e-03
   1.07896681e-03  4.72197209e-02  2.49630794e-03  7.34235065e-04
   3.96690334e-04  3.56127113e-03  1.81484438e-03  1.08004417e-04
   3.13525733e-02  9.71876997e-05  4.91171113e-04  1.18034275e-03
   8.16603942e-03  7.38250739e-04  1.96493932e-03  9.90657974e-01
   4.25603131e-04  5.02973499e-03  3.11216887e-04  9.34220376e-05
   1.29872316e-03  6.48466116e-04  8.85977961e-04  7.62763300e-03
   2.35088937e-03  8.28115611e-04  7.81901238e-04  1.74846514e-03
   1.08374694e-03  4.82318152e-02  2.83121262e-04  4.61383461e-03
   5.10166787e-03  9.16860601e-03  8.76713605e-04  1.27297635e-03
   1.49549185e-03  3.72199733e-04  8.85550987e-04  2.06190377e-03
 

In [21]:
terms = vectorizer.get_feature_names_out()
print(terms) # 벡터의 각 차원이 어떤 단어를 의미하는지 (Each dimension is matched with which word)
print(len(terms)) # 총 벡터의 차원은 100차원 (Dimension: 100)

['간지' '감성' '갑툭튀' '거북이' '겠냐' '고고씽' '고양이' '고진감래' '궁금' '귀여니' '기절' '꼬마' '꾸웨액'
 '꾸웨엑' '낭만' '너뭐돼' '농협은행' '뉴진스' '당모치' '대박' '댕댕이' '동료' '디토' '띵작' '레게노' '레알'
 '루돌프' '루삥뽕' '만남' '맑눈광' '맛꿀마' '머선' '먹방' '모에' '무야호' '문제' '뭥미' '부캐' '분좋카'
 '브라보' '빙고' '빵꾸똥꾸' '삼귀다' '소고기' '손민수' '쉽살재빙' '스라밸' '스불재' '신나' '신뢰' '쌉파서블'
 '아가씨' '아메리카노' '아이스' '알잘딱깔센' '어사' '어쩔티비' '억텐' '얼죽아' '얼짱' '엄마' '여신' '연진'
 '엽기' '오운완' '오이시쿠나레' '오타쿠' '우왕' '원영' '이유' '이제' '인성' '일취월장' '잡채' '재질' '잼민'
 '잼민이' '저메추' '저쩔' '점메추' '존귀' '졸귀' '중꺽마' '중꺾마' '지대' '지못미' '쫌쫌따리' '찐텐' '초딩'
 '추구미' '축복' '캘박' '커피' '킹받네' '킹받드라슈' '킹왕짱' '하이' '한강' '혜정' '홍대']
100


In [22]:
model.components_[0].shape

(100,)

- <span style = 'font-size:1.2em;line-height:1.5em'>각 topic에 대해서 상위 5개의 큰 값을 갖는 단어를 출력하는 함수</span>

In [23]:
def get_keyword_by_topic(components, feature_name, n=5):
    for idx, topic in enumerate(components):
        sorted_keyword_idx = np.argsort(components[idx])[-1:-n-1:-1]
        sorted_keyword_result = [(feature_name[x], topic[x].round(4)) for x in sorted_keyword_idx]
        print(f'Topic {idx}: {sorted_keyword_result}')

In [24]:
get_keyword_by_topic(model.components_, terms)

Topic 0: [('문제', 0.9907), ('이제', 0.071), ('엄마', 0.0517), ('신뢰', 0.0482), ('동료', 0.0472)]
Topic 1: [('엄마', 0.9283), ('감성', 0.1975), ('궁금', 0.1816), ('이제', 0.1467), ('만남', 0.1028)]
Topic 2: [('감성', 0.6693), ('궁금', 0.5728), ('이제', 0.2732), ('동료', 0.0957), ('만남', 0.0687)]
Topic 3: [('궁금', 0.7796), ('신뢰', 0.0334), ('동료', 0.0135), ('오타쿠', 0.0088), ('소고기', 0.0062)]
Topic 4: [('이제', 0.8103), ('동료', 0.4206), ('신뢰', 0.086), ('만남', 0.0673), ('고양이', 0.0477)]
Topic 5: [('동료', 0.8635), ('만남', 0.1849), ('감성', 0.0618), ('고양이', 0.0444), ('신뢰', 0.0296)]


# <b> 2. LDA(Latent Dirichlet Allocation)</b> 

In [25]:
# preprocessed_body에 있는 단어를 리스트에 넣은 칼럼 생성
df['word_list'] = df['preprocessed_inwords'].apply(lambda x: x.split())

In [26]:
df['word_list'].value_counts()

word_list
[]                                  1812
[문제]                                 224
[님]                                  184
[문제, 문제]                              83
[궁금]                                  70
                                    ... 
[킹, 커피]                                1
[문제, 문제, 이제, 님, 님]                     1
[궁금, 인성, 궁금, 문제, 인성, 궁금, 궁금, 인성]       1
[무야호, 무야호, 무야호, 무야호, 커피]               1
[엄마, 초딩, 님, 문제, 엄마, 엄마]                1
Name: count, Length: 2054, dtype: int64

In [27]:
lst = df['word_list'].apply(lambda words: [word for word in words if len(word) == 1])
lst
# word_list에서 한 글자인 단어 확인 -> 유행어는 한 글자인 유행어도 있기 때문에 삭제하지 않음

0                      [즐, 즐, 즐]
1                             []
2                             []
3                             []
4                             []
                  ...           
5624                          []
5625                          []
5626                         [님]
5627    [님, 님, 님, 님, 님, 님, 님, 님]
5628                         [님]
Name: word_list, Length: 5624, dtype: object

In [28]:
# # -> 유행어는 한 글자인 유행어도 있기 때문에 삭제하지 않음
# df['word_list'] = df['word_list'].apply(lambda words: [word for word in words if len(word) > 1])
# df['word_list'].value_counts()

In [29]:
from gensim import corpora

word_dict = corpora.Dictionary(df['word_list'])

In [30]:
corpus = [word_dict.doc2bow(text) for text in df['word_list']]

In [31]:
len(corpus)

5624

In [32]:
corpus[2]  ## 2번째 문서에 대해 각각의 단어들이 몇 번을 나왔는지.. 

[(5, 6)]

In [33]:
[(idx,word) for idx, word in word_dict.items()]

[(0, '레알'),
 (1, '엽기'),
 (2, '즐'),
 (3, '지대'),
 (4, '고고씽'),
 (5, '문제'),
 (6, '듣보잡'),
 (7, '레게노'),
 (8, '만남'),
 (9, '맛꿀마'),
 (10, '아메리카노'),
 (11, '아이스'),
 (12, '어쩔티비'),
 (13, '저메추'),
 (14, '킹'),
 (15, '킹왕짱'),
 (16, '궁금'),
 (17, '뷁'),
 (18, '빙고'),
 (19, '쉽살재빙'),
 (20, '알잘딱깔센'),
 (21, '하이'),
 (22, '기절'),
 (23, '꾸웨엑'),
 (24, '디토'),
 (25, '이제'),
 (26, '추구미'),
 (27, '폼'),
 (28, '거북이'),
 (29, '스불재'),
 (30, '당모치'),
 (31, '머선'),
 (32, '재질'),
 (33, '쫌쫌따리'),
 (34, '감성'),
 (35, '귀여니'),
 (36, '당빠지'),
 (37, '소고기'),
 (38, '우왕'),
 (39, '지못미'),
 (40, '루삥뽕'),
 (41, '먹방'),
 (42, '분좋카'),
 (43, '얼짱'),
 (44, '엄마'),
 (45, '점메추'),
 (46, '동료'),
 (47, '님'),
 (48, '캘박'),
 (49, '킹받네'),
 (50, '킹받드라슈'),
 (51, '갑분싸'),
 (52, '대박'),
 (53, '뭥미'),
 (54, '빵꾸똥꾸'),
 (55, '손민수'),
 (56, '엄친아'),
 (57, '여신'),
 (58, '오타쿠'),
 (59, '완소'),
 (60, '쩔어'),
 (61, '초딩'),
 (62, '억텐'),
 (63, '탄'),
 (64, '낭만'),
 (65, '잡채'),
 (66, '커피'),
 (67, '삼귀다'),
 (68, '잼민'),
 (69, '모에모에'),
 (70, '오이시쿠나레'),
 (71, '큥'),
 (72, '뀽'),
 (73, '부캐'),
 (74, '찐

## LDA 모델 훈련 (Topic 수: 6개로 설정)

In [34]:
import gensim

In [35]:
N_TOPICS = 6
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics = N_TOPICS,  ## 주제 몇개할 건지
                                           id2word=word_dict)   ## 보카뷸러리 셋

### 각 topic의 대표 단어들을 출력

In [36]:
topics = ldamodel.print_topics(num_words=5)  ## 각 토픽들에 대해서 몇 개의 단어(num_words)로 표현할 것인지
## 대표적인 단어를 알 수 있음
for topic in topics:
    print(topic)

(0, '0.520*"문제" + 0.079*"고양이" + 0.069*"신뢰" + 0.058*"폼" + 0.029*"동료"')
(1, '0.279*"궁금" + 0.080*"뉴진스" + 0.072*"낭만" + 0.069*"디토" + 0.069*"부캐"')
(2, '0.194*"만남" + 0.112*"좌" + 0.084*"먹방" + 0.083*"동료" + 0.037*"원영"')
(3, '0.694*"님" + 0.048*"축복" + 0.030*"짱" + 0.027*"동료" + 0.022*"하이"')
(4, '0.412*"엄마" + 0.094*"이제" + 0.039*"문제" + 0.033*"잡채" + 0.032*"님"')
(5, '0.242*"감성" + 0.225*"킹" + 0.112*"한강" + 0.067*"홍대" + 0.039*"커피"')


## LDA 시각화

In [47]:
from IPython.core.display import HTML
import pyLDAvis
from pyLDAvis import gensim_models

In [48]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, word_dict)
pyLDAvis.display(vis)

## 문서 별 Topic 분포

In [49]:
for i, topic_list in enumerate(ldamodel[corpus]):
    print(f'{i}번째 문서의 topic 비중: {topic_list}')
    if i==5:
        break

0번째 문서의 topic 비중: [(0, 0.40880507), (4, 0.4624078), (5, 0.10602154)]
1번째 문서의 topic 비중: [(0, 0.08352016), (1, 0.58305466), (2, 0.08333664), (3, 0.083376385), (4, 0.08334103), (5, 0.083371125)]
2번째 문서의 topic 비중: [(0, 0.023820346), (1, 0.023822233), (2, 0.02384291), (3, 0.8808904), (4, 0.023812562), (5, 0.023811553)]
3번째 문서의 topic 비중: [(0, 0.16666667), (1, 0.16666667), (2, 0.16666667), (3, 0.16666667), (4, 0.16666667), (5, 0.16666667)]
4번째 문서의 topic 비중: [(0, 0.16666667), (1, 0.16666667), (2, 0.16666667), (3, 0.16666667), (4, 0.16666667), (5, 0.16666667)]
5번째 문서의 topic 비중: [(0, 0.16666667), (1, 0.16666667), (2, 0.16666667), (3, 0.16666667), (4, 0.16666667), (5, 0.16666667)]


### 각 문서에 대해서 Topic의 비중을 나타내는 함수 작성

In [50]:
## 교수님 코드 

def get_topic_ratio_for_each_document(ldamodel, corpus):
    results = []
    for i, topic_list in enumerate(ldamodel[corpus]):
        sorted_topic_list = sorted(topic_list, key = lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list]
        results.append(doc_result)
        
    results = pd.DataFrame(results, columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])
    return results

In [51]:
df_results = get_topic_ratio_for_each_document(ldamodel, corpus)

df_results

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,4,0.462412,"[(4, 0.46241218), (0, 0.4087967), (5, 0.106025..."
1,1,1,0.583054,"[(1, 0.58305407), (0, 0.08352077), (3, 0.08337..."
2,2,3,0.880889,"[(3, 0.8808889), (2, 0.02384437), (1, 0.023822..."
3,3,0,0.166667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."
4,4,0,0.166667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."
...,...,...,...,...
5619,5619,0,0.166667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."
5620,5620,0,0.166667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."
5621,5621,3,0.459920,"[(3, 0.45991984), (1, 0.2872815), (4, 0.148159..."
5622,5622,0,0.642516,"[(0, 0.64251643), (3, 0.30614847), (2, 0.01284..."


In [52]:
## 참고 사이트: https://wikidocs.net/30708

def make_topictable_per_doc(ldamodel, corpus):
    # 데이터를 저장할 빈 리스트 초기화
    topic_data = []

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_data.append([i, int(topic_num), round(prop_topic, 4), topic_list])
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
            
    # 리스트를 데이터프레임으로 변환
    topic_table = pd.DataFrame(topic_data, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])

    return(topic_table)

In [53]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable[:10]

,index,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0,4,0.4624,"[(0, 0.4088374), (4, 0.46238902), (5, 0.106008)]"
1,1,1,1,0.5831,"[(0, 0.08352049), (1, 0.58305436), (2, 0.08333..."
2,2,2,3,0.8809,"[(0, 0.023820354), (1, 0.023822242), (2, 0.023..."
3,3,3,0,0.1667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."
4,4,4,0,0.1667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."
5,5,5,0,0.1667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."
6,6,6,4,0.4707,"[(0, 0.015192885), (1, 0.4681438), (2, 0.01549..."
7,7,7,0,0.1667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."
8,8,8,4,0.5202,"[(0, 0.024086524), (1, 0.023824126), (2, 0.023..."
9,9,9,0,0.1667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666..."


In [54]:
## 교수님 코드 + 각 토픽 별 키워드를 볼 수 있는 함수 (GPT의 도움을 조금 받음!)


def get_topic_keywords(ldamodel, num_keywords=5):
    topic_keywords = {}
    for topic_id in range(ldamodel.num_topics):
        keywords = [word for word, _ in ldamodel.show_topic(topic_id, topn=num_keywords)]
        topic_keywords[topic_id] = keywords
    return topic_keywords

def get_topic_ratio_for_each_document(ldamodel, corpus):
    topic_keywords = get_topic_keywords(ldamodel)
    results = []
    for i, topic_list in enumerate(ldamodel[corpus]):
        sorted_topic_list = sorted(topic_list, key = lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        second_most_important_topic, second_most_important_ratio = sorted_topic_list[1] if len(sorted_topic_list) > 1 else (None, 0)

        keywords_1 = ", ".join(topic_keywords[most_important_topic])
        keywords_2 = ", ".join(topic_keywords[second_most_important_topic]) if second_most_important_topic is not None else ''
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list, keywords_1, keywords_2]
        results.append(doc_result)
        
    results = pd.DataFrame(results, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중', 'Top1_Topic_keyword', 'Top2_Topic_keyword'])
    return results

In [55]:
df_results = get_topic_ratio_for_each_document(ldamodel, corpus)

df_results

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중,Top1_Topic_keyword,Top2_Topic_keyword
0,0,4,0.462418,"[(4, 0.46241766), (0, 0.40878293), (5, 0.10603...","고양이, 탄, 부캐, 커피, 재질","님, 오타쿠, 고양이, 문제, 킹"
1,1,1,0.583054,"[(1, 0.58305424), (0, 0.083520584), (3, 0.0833...","엄마, 만남, 동료, 감성, 님","님, 오타쿠, 고양이, 문제, 킹"
2,2,3,0.880890,"[(3, 0.8808901), (2, 0.023843253), (1, 0.02382...","문제, 이제, 감성, 신뢰, 축복","폼, 킹, 낭만, 님, 좌"
3,3,0,0.166667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666...","님, 오타쿠, 고양이, 문제, 킹","엄마, 만남, 동료, 감성, 님"
4,4,0,0.166667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666...","님, 오타쿠, 고양이, 문제, 킹","엄마, 만남, 동료, 감성, 님"
...,...,...,...,...,...,...
5619,5619,0,0.166667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666...","님, 오타쿠, 고양이, 문제, 킹","엄마, 만남, 동료, 감성, 님"
5620,5620,0,0.166667,"[(0, 0.16666667), (1, 0.16666667), (2, 0.16666...","님, 오타쿠, 고양이, 문제, 킹","엄마, 만남, 동료, 감성, 님"
5621,5621,3,0.459958,"[(3, 0.45995787), (1, 0.28719193), (4, 0.14817...","문제, 이제, 감성, 신뢰, 축복","엄마, 만남, 동료, 감성, 님"
5622,5622,0,0.642512,"[(0, 0.6425122), (3, 0.30615267), (2, 0.012840...","님, 오타쿠, 고양이, 문제, 킹","문제, 이제, 감성, 신뢰, 축복"
